In [1]:
# For dealing with files
import os
import shutil
from pathlib import Path
# For saving data matrix
import numpy as np

# For normalising data
from statistics import pstdev,mean
# For using regex expressions
import re

In [2]:
# save where you currently are (for coming back afterwards)
#old_path = path('C:\Users\bjorn\Documents\Repositories\deep-machine-learning\Project')
old_path = os.getcwd()
print(old_path)
# cd to sequencec directory
os.chdir('sequences')
'''# Get a list of all filenames inside
files = os.listdir()
print(files[0:10])
# Filter list using regex expressions
r_testimg_inputdata = re.compile('testimg.*.inputdata.txt')
r_trainimg_inputdata = re.compile('trainimg.*.inputdata.txt')
print(type(r_testimg_inputdata))
#all_train_filenames = list(filter(r_trainimg_inputdata.match, files))
#all_test_filenames = list(filter(r_testimg_inputdata.match, files))'''

# Fulhax galore pga pallar inte mer
prefix_train='trainimg-'
prefix_test='testimg-'
suffix='-inputdata.txt'
all_train_filenames = [prefix_train+str(i)+suffix for i in range(60000)]
all_test_filenames = [prefix_test+str(i)+suffix for i in range(10000)]
print(all_train_filenames[0:10])
'''for counter, file in enumerate(all_train_filenames):
    tmp_length=len(open(file, 'r').readlines())
    if max_sequence_length<tmp_length:
        max_sequence_length=tmp_length
print(max_sequence_length)'''

def preproces_data_relative_coord(filenames):
    # pre-allocate data matrix
    nbrSequences=len(filenames)
    max_sequence_length=117
    nbr_features = 4      
    data = np.zeros((nbrSequences,max_sequence_length,nbr_features),dtype=np.int8)
    for sequnce_ix, file in enumerate(filenames):
        list_of_sequence=open(file, 'r').readlines()
        length_sequence = len(list_of_sequence)
        if sequnce_ix%1500==0:
            print(sequnce_ix)
        for time_ix,row in enumerate(list_of_sequence):
            (dx,dy,eos,eod) = [int(i) for i in row.split(' ')]
            data[sequnce_ix][-length_sequence+time_ix]=[dx,dy,eos,eod]
    return data
            
def preproces_data_absolute_coord(filenames):
    # pre-allocate data matrix
    nbrSequences=len(filenames)
    max_sequence_length=117
    nbr_features = 4      
    data = np.zeros((nbrSequences,max_sequence_length,nbr_features),dtype=np.int8)
    for sequnce_ix, file in enumerate(filenames):
        list_of_sequence=open(file, 'r').readlines()
        length_sequence = len(list_of_sequence)
        if sequnce_ix%1500==0:
            print(sequnce_ix)
        tmp=np.empty((length_sequence,nbr_features))
        for time_ix,row in enumerate(list_of_sequence):
            (dx,dy,eos,eod) = [int(i) for i in row.split(' ')]
            tmp[time_ix]=[dx,dy,eos,eod]
        tmp=np.cumsum(tmp,axis=0) # Transform to absolute coordinates
        tmp_mean_x = mean(tmp[:,0])
        tmp_mean_y = mean(tmp[:,1])
        tmp[:,0] -= tmp_mean_x
        tmp[:,1] -= tmp_mean_y
        data[sequnce_ix][-length_sequence:]=tmp.copy()
    return data

test_data_rel_coord = preproces_data_relative_coord(all_test_filenames)
train_data_rel_coord = preproces_data_relative_coord(all_train_filenames) 
test_data_abs_coord = preproces_data_absolute_coord(all_test_filenames)
train_data_abs_coord = preproces_data_absolute_coord(all_train_filenames) 
os.chdir(old_path)

C:\Users\bjorn\Documents\Repositories\DeepMachineLearning\Project
['trainimg-0-inputdata.txt', 'trainimg-1-inputdata.txt', 'trainimg-2-inputdata.txt', 'trainimg-3-inputdata.txt', 'trainimg-4-inputdata.txt', 'trainimg-5-inputdata.txt', 'trainimg-6-inputdata.txt', 'trainimg-7-inputdata.txt', 'trainimg-8-inputdata.txt', 'trainimg-9-inputdata.txt']
0
1500
3000
4500
6000
7500
9000
0
1500
3000
4500
6000
7500
9000
10500
12000
13500
15000
16500
18000
19500
21000
22500
24000
25500
27000
28500
30000
31500
33000
34500
36000
37500
39000
40500
42000
43500
45000
46500
48000
49500
51000
52500
54000
55500
57000
58500
0
1500
3000
4500
6000
7500
9000
0
1500
3000
4500
6000
7500
9000
10500
12000
13500
15000
16500
18000
19500
21000
22500
24000
25500
27000
28500
30000
31500
33000
34500
36000
37500
39000
40500
42000
43500
45000
46500
48000
49500
51000
52500
54000
55500
57000
58500


In [3]:
np.save('test_data_rel_coord.npy',test_data_rel_coord)
np.save('train_data_rel_coord.npy',train_data_rel_coord)
np.save('test_data_abs_coord.npy',test_data_abs_coord)
np.save('train_data_abs_coord.npy',train_data_abs_coord)